In [1]:
!pip3 install seaborn

In [2]:
!pip3 install scikit-learn

In [3]:
!pip3 install --upgrade scikit-learn

In [4]:
!pip3 install imbalanced-learn

In [5]:
!pip3 install xgboost

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
import sklearn.preprocessing as preproc
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_pickle('shared/Project-3_NYC_311_Calls.pkl')
df = df.set_index(pd.DatetimeIndex(df['Created Date']))
del df['Created Date']

In [3]:
df.head()

,Unique Key,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,City,Resolution Description,Borough,Open Data Channel Type
Created Date,,,,,,,,,,,
2011-04-06 00:00:00,20184537,HPD,Department of Housing Preservation and Develop...,HEATING,HEAT,RESIDENTIAL BUILDING,10002.0,NEW YORK,More than one complaint was received for this ...,MANHATTAN,UNKNOWN
2011-04-06 00:00:00,20184538,HPD,Department of Housing Preservation and Develop...,GENERAL CONSTRUCTION,WINDOWS,RESIDENTIAL BUILDING,11236.0,BROOKLYN,The Department of Housing Preservation and Dev...,BROOKLYN,UNKNOWN
2011-04-06 00:00:00,20184539,HPD,Department of Housing Preservation and Develop...,PAINT - PLASTER,WALLS,RESIDENTIAL BUILDING,10460.0,BRONX,The Department of Housing Preservation and Dev...,BRONX,UNKNOWN
2022-07-08 11:14:43,54732265,DSNY,Department of Sanitation,Dirty Condition,Trash,Sidewalk,10467.0,BRONX,The Department of Sanitation investigated this...,BRONX,PHONE
2011-04-06 00:00:00,20184540,HPD,Department of Housing Preservation and Develop...,NONCONST,VERMIN,RESIDENTIAL BUILDING,10460.0,BRONX,The Department of Housing Preservation and Dev...,BRONX,UNKNOWN


In [4]:
df_subset = df[['Unique Key']]

In [5]:
df_subset.head()

,Unique Key
Created Date,
2011-04-06 00:00:00,20184537
2011-04-06 00:00:00,20184538
2011-04-06 00:00:00,20184539
2022-07-08 11:14:43,54732265
2011-04-06 00:00:00,20184540


In [6]:
df_subset.describe()

,Unique Key
count,3.378098e+07
mean,3.802665e+07
std,1.209574e+07
min,1.056422e+07
25%,2.793121e+07
50%,3.798303e+07
75%,4.830951e+07
max,5.840682e+07


In [7]:
df_subset.dropna()

,Unique Key
Created Date,
2011-04-06 00:00:00,20184537
2011-04-06 00:00:00,20184538
2011-04-06 00:00:00,20184539
2022-07-08 11:14:43,54732265
2011-04-06 00:00:00,20184540
...,...
2011-04-06 00:00:00,20184532
2011-04-06 00:00:00,20184533
2011-04-06 00:00:00,20184534


In [8]:
df_daily = df_subset.resample('D').count()

In [9]:
df_daily.head()

,Unique Key
Created Date,
2010-01-01,2942
2010-01-02,3958
2010-01-03,5676
2010-01-04,9763
2010-01-05,8735


In [10]:
average_daily_complaints_2022 = df_daily[df_daily.index.year == 2022].mean()
print(average_daily_complaints_2022)

Unique Key    8684.320548
dtype: float64


In [11]:
date_max_calls = df_daily.idxmax()
print(date_max_calls)

Unique Key   2020-08-04
dtype: datetime64[ns]


In [12]:
df_max_calls = df[df.index.date == date_max_calls[0].date()]
most_common_complaint_max_calls = df_max_calls['Complaint Type'].mode().iloc[0]
print(most_common_complaint_max_calls)

Damaged Tree


In [13]:
quietest_month = df['Unique Key'].resample('M').count().idxmin().strftime('%B %Y')
print(quietest_month)


August 2023


In [14]:
df_daily = df['Unique Key'].resample('D').count()

In [15]:
!pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable


In [50]:
!pip install Prophet

Defaulting to user installation because normal site-packages is not writeable


In [16]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df_daily, model='additive')
seasonal_component_dec_25 = result.seasonal['2020-12-25']

In [17]:
print(seasonal_component_dec_25)

182.69763790386236


In [18]:
autocorr_lag1 = df_daily.autocorr(lag=1)

In [19]:
print(autocorr_lag1)

0.7517059728398577


In [52]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error

In [58]:
prophet_df = df_daily.reset_index()
prophet_df.columns = ['ds', 'y']

In [59]:
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])

In [60]:
train = prophet_df.iloc[:-90]
test = prophet_df.iloc[-90:]

In [61]:
model = Prophet()
model.fit(train)

17:44:37 - cmdstanpy - INFO - Chain [1] start processing
17:44:38 - cmdstanpy - INFO - Chain [1] done processing


In [62]:
future = model.make_future_dataframe(periods=90)
forecast = model.predict(future)

test_forecast = forecast.set_index('ds').loc[test['ds']]
rmse = np.sqrt(mean_squared_error(test['y'], test_forecast['yhat']))
print(rmse)

1231.513760758433
